<article class="day-desc"><h2>--- Day 4: Printing Department ---</h2><p>You ride the escalator down to the printing department. They're clearly getting ready for Christmas; they have lots of large rolls of paper everywhere, and there's even a massive printer in the corner (to handle the really <span title="This joke is stupid and I love it.">big</span> print jobs).</p>
<p>Decorating here will be easy: they can make their own decorations. What you really need is a way to get further into the North Pole base while the elevators are offline.</p>
<p>"Actually, maybe we can help with that," one of the Elves replies when you ask for help. "We're pretty sure there's a cafeteria on the other side of the back wall. If we could break through the wall, you'd be able to keep moving. It's too bad all of our forklifts are so busy moving those big rolls of paper around."</p>
<p>If you can optimize the work the forklifts are doing, maybe they would have time to spare to break through the wall.</p>
<p>The rolls of paper (<code>@</code>) are arranged on a large grid; the Elves even have a helpful diagram (your puzzle input) indicating where everything is located.</p>
<p>For example:</p>
<pre><code>..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
</code></pre>
<p>The forklifts can only access a roll of paper if there are <em>fewer than four rolls of paper</em> in the eight adjacent positions. If you can figure out which rolls of paper the forklifts can access, they'll spend less time looking and more time breaking down the wall to the cafeteria.</p>
<p>In this example, there are <code><em>13</em></code> rolls of paper that can be accessed by a forklift (marked with <code>x</code>):</p>
<pre><code>..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
</code></pre>
<p>Consider your complete diagram of the paper roll locations. <em>How many rolls of paper can be accessed by a forklift?</em></p>
</article>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from aocd import get_data

puzzle_input = get_data(day=4, year=2025)
puzzle_input[:20]

'@@@@@.@@@@@@@.@.@@@.'

In [3]:
sample_input = """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@."""

sample_input

'..@@.@@@@.\n@@@.@.@.@@\n@@@@@.@.@@\n@.@@@@..@.\n@@.@@@@.@@\n.@@@@@@@.@\n.@.@.@.@@@\n@.@@@.@@@@\n.@@@@@@@@.\n@.@.@@@.@.'

In [4]:
from dataclasses import dataclass
from itertools import product
from typing import Self

DIRECTIONS = [
    (-1, 0),
    (1, 0),
    (0, -1),
    (0, 1),
    (-1, -1),
    (-1, 1),
    (1, -1),
    (1, 1),
]


@dataclass(frozen=True, slots=True)
class Grid:
    cells: tuple[tuple[str, ...], ...]

    @classmethod
    def from_string(cls, diagram: str) -> Self:
        return cls(tuple(tuple(line) for line in diagram.splitlines()))

    @property
    def rows(self) -> int:
        return len(self.cells)

    @property
    def cols(self) -> int:
        return len(self.cells[0])

    def is_in_bounds(self, r: int, c: int) -> bool:
        return 0 <= r < self.rows and 0 <= c < self.cols

    def is_roll(self, r: int, c: int) -> bool:
        return self.cells[r][c] == "@"

    def count_neighboring_rolls(self, r: int, c: int) -> int:
        return sum(
            1
            for dr, dc in DIRECTIONS
            if self.is_in_bounds(r + dr, c + dc) and self.is_roll(r + dr, c + dc)
        )

    def is_accessible(self, r: int, c: int) -> bool:
        return self.is_roll(r, c) and self.count_neighboring_rolls(r, c) < 4

    def accessible_positions(self) -> frozenset[tuple[int, int]]:
        return frozenset(
            (r, c)
            for r, c in product(range(self.rows), range(self.cols))
            if self.is_accessible(r, c)
        )

    def count_accessible(self) -> int:
        return len(self.accessible_positions())

    def remove_accessible_rolls(self) -> Self:
        """Return a new Grid with accessible rolls removed."""
        positions = self.accessible_positions()
        return type(self)(
            tuple(
                tuple(
                    "." if (r, c) in positions else cell for c, cell in enumerate(row)
                )
                for r, row in enumerate(self.cells)
            )
        )


Grid.from_string(sample_input).count_accessible()

13

In [5]:
part1 = Grid.from_string(puzzle_input).count_accessible()
part1

1502

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, the Elves just need help accessing as much of the paper as they can.</p>
<p>Once a roll of paper can be accessed by a forklift, it can be <em>removed</em>. Once a roll of paper is removed, the forklifts might be able to access <em>more</em> rolls of paper, which they might also be able to remove. How many total rolls of paper could the Elves remove if they keep repeating this process?</p>
<p>Starting with the same example as above, here is one way you could remove as many rolls of paper as possible, using highlighted <code><em>@</em></code> to indicate that a roll of paper is about to be removed, and using <code>x</code> to indicate that a roll of paper was just removed:</p>
<pre><code>Initial state:
..<em>@</em><em>@</em>.<em>@</em><em>@</em>@<em>@</em>.
<em>@</em>@@.@.@.@@
@@@@@.<em>@</em>.@@
@.@@@@..@.
<em>@</em>@.@@@@.@<em>@</em>
.@@@@@@@.@
.@.@.@.@@@
<em>@</em>.@@@.@@@@
.@@@@@@@@.
<em>@</em>.<em>@</em>.@@@.<em>@</em>.

Remove 13 rolls of paper:
..xx.xx<em>@</em>x.
x@@.<em>@</em>.<em>@</em>.@<em>@</em>
<em>@</em>@@@@.x.@@
<em>@</em>.@@@@..<em>@</em>.
x@.@@@@.<em>@</em>x
.<em>@</em>@@@@@@.<em>@</em>
.<em>@</em>.@.@.@@@
x.@@@.@@@@
.<em>@</em>@@@@@@@.
x.x.@@@.x.

Remove 12 rolls of paper:
.......x..
.<em>@</em>@.x.x.<em>@</em>x
x@@@@...<em>@</em><em>@</em>
x.@@@@..x.
.<em>@</em>.@@@@.x.
.x@@@@@@.x
.x.@.@.@@<em>@</em>
..@@@.@@@@
.x<em>@</em>@@@@@@.
....@@@...

Remove 7 rolls of paper:
..........
.x<em>@</em>.....x.
.<em>@</em>@@@...xx
..@@@@....
.x.@@@@...
..<em>@</em>@@@@@..
...@.@.@@x
..<em>@</em>@@.@@@<em>@</em>
..x@@@@@@.
....@@@...

Remove 5 rolls of paper:
..........
..x.......
.x<em>@</em>@@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@<em>@</em>.
....@@@...

Remove 2 rolls of paper:
..........
..........
..x@@.....
..<em>@</em>@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...

Remove 1 roll of paper:
..........
..........
...<em>@</em>@.....
..x@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
...x<em>@</em>.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
....x.....
...<em>@</em>@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...
</code></pre>
<p>Stop once no more rolls of paper are accessible by a forklift. In this example, a total of <code><em>43</em></code> rolls of paper can be removed.</p>
<p>Start with your original diagram. <em>How many rolls of paper in total can be removed by the Elves and their forklifts?</em></p>
</article>

In [6]:
from typing import Iterator


def removal_sequence(grid: Grid) -> Iterator[Grid]:
    """Yield grids with accessible rolls, stopping when none are accessible."""
    while grid.count_accessible() > 0:
        yield grid
        grid = grid.remove_accessible_rolls()


def total_removable(diagram: str) -> int:
    return sum(
        g.count_accessible() for g in removal_sequence(Grid.from_string(diagram))
    )


total_removable(sample_input)

43

In [7]:
part2 = total_removable(puzzle_input)
part2

9083